# Interpretability of argument impact

In [ ]:
!pip install -U numpy
!pip install transformers==4.22.1

# 注意！运行完这个cell, 点击"代码执行程序"→"重新启动代码执行程序"， 接着从下面的cell开始运行就可以

In [ ]:
!pip install ecco

## 这里是这个包支持的可解释性方法。选择一个就可以。下面的例子里用了'grad_x_input'

ATTR_NAME_ALIASES = {
    'ig': 'integrated_gradients',
    'saliency': 'gradient',
    'dl': 'deep_lift',
    'dls': 'deep_lift_shap',
    'gb': 'guided_backprop',
    'gg': 'guided_gradcam',
    'deconv': 'deconvolution',
    'lrp': 'layer_relevance_propagation'
}

ATTR_NAME_TO_CLASS = { # TODO: Add more Captum Primary attributions with needed computed arguments
    'integrated_gradients': IntegratedGradients,
    'gradient': Saliency,
    'grad_x_input': InputXGradient,
    'deep_lift': DeepLift,
    'deep_lift_shap': DeepLiftShap,
    'guided_backprop': GuidedBackprop,
    'guided_gradcam': GuidedGradCam,
    'deconvolution': Deconvolution,
    'layer_relevance_propagation': LRP
}

## 1. no_knowledge model

In [38]:
import ecco

hf_model_id = 'JoeyCheng/flan_t5_base_argmining_no_knowledge'

model_config = {
    'embedding': 'shared.weight',
    'type': 'enc-dec',
    'activations': 'wo',
    'token_prefix': '▁',
    'partial_token_prefix': ''
}

ecco_model = ecco.from_pretrained(hf_model_id, activations=True, model_config=model_config)

### 1.1 Example1: idx=0 from test set. ture_label=IMPACTFUL

In [39]:
input="""Context: Cloning animals is ethical The possible scientific advancements outweigh the ethical issues. Argument: Cloning animals advances science in that
scientists learn better about what cloning effects there are. The impact is"""

# output = ecco_model.generate(review, generate=5, do_sample=False)
output = ecco_model.generate(input, generate=5, do_sample=False, attribution=['grad_x_input'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
output.primary_attributions(attr_method='grad_x_input', ignore_tokens=[0,1,2,3,4,5,6,43,44])

<IPython.core.display.Javascript object>

### 1.2 Example2: idx=1 from test set. ture_label=MEDIUM IMPACT

In [41]:
input="""Context: Europeans, defined as citizens of what today constitutes the European Union (EU), had a better quality of life in the 1990s, than in the 1890s. In the
1990s people were much easier to influence than in 1890s, due to the large reach of media such as the radio or the television. Improvements in communication
technology increased the use of propaganda. Argument: Propaganda was widely used before the 20th century already. The impact is"""

# output = ecco_model.generate(review, generate=5, do_sample=False)
output = ecco_model.generate(input, generate=5, do_sample=False, attribution=['grad_x_input'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
output.primary_attributions(attr_method='grad_x_input', ignore_tokens=[0,1,2,3,4,5,6,43,44])

<IPython.core.display.Javascript object>

### 1.3 Example3: idx=2 from test set. ture_label=NOT IMPACTFUL

In [43]:
input="""Context: Polygamy should be legal Polygamy can have positive consequences for society. Argument: Polygamous marriages bear a high likelihood of early widowhood.
The impact is"""

# output = ecco_model.generate(review, generate=5, do_sample=False)
output = ecco_model.generate(input, generate=5, do_sample=False, attribution=['grad_x_input'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
output.primary_attributions(attr_method='grad_x_input', ignore_tokens=[0,1,2,3,4,5,6])

<IPython.core.display.Javascript object>

## 2. with_knowledge model

In [45]:
import ecco

hf_model_id = 'JoeyCheng/flan_t5_base_argmining_knowledge'

model_config = {
    'embedding': 'shared.weight',
    'type': 'enc-dec',
    'activations': 'wo',
    'token_prefix': '▁',
    'partial_token_prefix': ''
}

ecco_model = ecco.from_pretrained(hf_model_id, activations=True, model_config=model_config)

### 2.1 Example1: idx=0 from test set. ture_label=IMPACTFUL

In [46]:
review="""Persona Knowledge: Role: Scientist; Stance: Pro; Argument: Cloning animals can lead to significant scientific advancements, such as understanding how genes
affect behavior and health. The ethical concerns are outweighed by the potential benefits; Character traits: Rational, objective, curious; Intent: To contribute
to scientific research and knowledge. Role: Animal rights activist; Stance: Con; Argument: Cloning animals is unethical as it involves manipulating and
exploiting living creatures. The potential scientific advancements do not justify the harm caused to animals; Character traits: Compassionate, passionate,
idealistic; Intent: To protect the welfare of animals and prevent their mistreatment. Role: Bioethicist; Stance: Neutral; Argument: Cloning animals raises
important ethical questions regarding the moral status of animals and their treatment. The potential scientific advancements must be weighed against the ethical
implications; Character traits: Objective, analytical, critical; Intent: To analyze and evaluate the ethical considerations of cloning animals from a neutral
perspective. Role: Farmer; Stance: Pro; Argument: Cloning animals can lead to more efficient and cost-effective breeding practices, resulting in higher yields
and profits. The ethical concerns are outweighed by the potential economic benefits; Character traits: Pragmatic, resourceful, business-oriented; Intent: To
improve their farming practices and increase their profits. Role: Consumer; Stance: Neutral; Argument: Cloning animals may result in cheaper and more accessible
meat products, but the potential ethical concerns must be taken into account when making purchasing decisions; Character traits: Practical, cost-conscious,
health-conscious; Intent: To make informed decisions as a consumer and balance ethical concerns with personal preferences. Role: Religious leader; Stance: Con;
Argument: Cloning animals is playing God and goes against the natural order of things. It is unethical to manipulate and create life in this way; Character
traits: Traditional, moralistic, spiritual; Intent: To uphold religious values and principles and prevent the violation of ethical boundaries. Context: Cloning
animals is ethical The possible scientific advancements outweigh the ethical issues. Argument: Cloning animals advances science in that scientists learn better
about what cloning effects there are. The impact is"""

# output = ecco_model.generate(review, generate=5, do_sample=False, attribution=['ig'])
output = ecco_model.generate(review, generate=5, do_sample=False, attribution=['grad_x_input'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
output.primary_attributions(attr_method='grad_x_input', ignore_tokens=[0,1,2,3,4,5,6,43,44])

<IPython.core.display.Javascript object>

### 2.2 Example2: idx=1 from test set. ture_label=MEDIUM IMPACT

In [48]:
input="""Persona Knowledge: Role: Historian; Stance: Pro; Argument: Propaganda has been used throughout history to influence public opinion and shape political
discourse. The use of propaganda in the 20th century was not a new phenomenon, but rather an evolution of previous techniques; Character traits: Knowledgeable,
analytical, objective; Intent: To provide historical context and perspective on the use of propaganda. Role: Journalist; Stance: Con; Argument: The use of
propaganda in the 20th century was more widespread and sophisticated than in previous centuries, due to advancements in technology and the rise of totalitarian
regimes. The manipulation of information and the use of propaganda undermined the freedom of the press and the public's trust in the media; Character traits:
Investigative, objective, ethical; Intent: To expose the dangers of propaganda and defend the principles of journalism. Role: Sociologist; Stance: Neutral;
Argument: The use of propaganda in the 20th century was a reflection of the social, political, and economic conditions of the time. The impact of propaganda on
society is complex and depends on various factors such as media literacy, political ideology, and cultural values; Character traits: Analytical, curious, open-
minded; Intent: To study the social phenomenon of propaganda and its effects on different groups in society. Role: Political strategist; Stance: Pro; Argument:
The use of propaganda in the 20th century was an effective tool for political campaigns and messaging. Propaganda techniques such as emotional appeals and
repetition can sway public opinion and mobilize voters; Character traits: Strategic, persuasive, competitive; Intent: To apply propaganda techniques to
political campaigns and gain an advantage over opponents. Role: Media executive; Stance: Neutral; Argument: The use of propaganda in the 20th century was a
reflection of the media industry's goals and practices. The media's role in disseminating information and shaping public opinion is complex and influenced by
various factors such as ownership, funding, and audience preferences; Character traits: Business-minded, pragmatic, adaptable; Intent: To navigate the media
landscape and achieve organizational objectives while maintaining journalistic integrity. Context: Europeans, defined as citizens of what today constitutes the
European Union (EU), had a better quality of life in the 1990s, than in the 1890s. In the 1990s people were much easier to influence than in 1890s, due to the
large reach of media such as the radio or the television. Improvements in communication technology increased the use of propaganda. Argument: Propaganda was
widely used before the 20th century already. The impact is"""

# output = ecco_model.generate(review, generate=5, do_sample=False)
output = ecco_model.generate(input, generate=5, do_sample=False, attribution=['grad_x_input'])

Token indices sequence length is longer than the specified maximum sequence length for this model (567 > 512). Running this sequence through the model will result in indexing errors


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
output.primary_attributions(attr_method='grad_x_input', ignore_tokens=[0,1,2,3,4,5,6,43,44])

<IPython.core.display.Javascript object>

### 2.3 Example3: idx=2 from test set. ture_label=NOT IMPACTFUL

In [50]:
input="""Persona Knowledge: Role: Feminist activist; Stance: Con; Argument: Polygamy is often used as a way to subjugate women, who are forced into marriages against
their will. This can lead to exploitation, abuse, and social isolation. Additionally, legalizing polygamy would only further perpetuate gender inequality;
Character traits: Empathetic, outspoken, principled; Intent: To advocate for the rights and empowerment of women. Role: Religious leader; Stance: Pro; Argument:
Polygamy is a religious practice that is recognized in certain cultures. Legalizing polygamy would allow individuals to practice their faith freely and could
strengthen family ties and community bonds; Character traits: Devout, traditional, persuasive; Intent: To defend the religious and cultural practices of their
community. Role: Social worker; Stance: Con; Argument: Polygamy can lead to economic and emotional instability, particularly for women and children.
Additionally, it can create legal and financial complications around inheritance, custody, and property rights; Character traits: Compassionate, practical,
resourceful; Intent: To protect vulnerable individuals and families from harm. Role: Lawyer; Stance: Neutral; Argument: Legalizing polygamy would require
significant changes to the legal system, particularly around marriage, divorce, and child custody. Additionally, it could raise questions around individual
rights, discrimination, and the definition of marriage; Character traits: Analytical, strategic, impartial; Intent: To provide legal advice and navigate the
complex legal implications of polygamy. Role: Anthropologist; Stance: Pro; Argument: Polygamy is a common practice in many societies and has historically been
used to strengthen family bonds, increase social status, and ensure economic stability. Legalizing polygamy would allow individuals to practice their cultural
traditions freely; Character traits: Objective, curious, open-minded; Intent: To study and understand the cultural practices of different societies, and promote
cross-cultural awareness and understanding. Context: Polygamy should be legal Polygamy can have positive consequences for society. Argument: Polygamous
marriages bear a high likelihood of early widowhood. The impact is"""

# output = ecco_model.generate(review, generate=5, do_sample=False)
output = ecco_model.generate(input, generate=5, do_sample=False, attribution=['grad_x_input'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
output.primary_attributions(attr_method='grad_x_input', ignore_tokens=[0,1,2,3,4,5,6,43,44])

<IPython.core.display.Javascript object>